In [1]:
import numpy as np
import scipy.stats as st

# Function to generate GBM price paths
def asset_paths_gbm(S0, r, sigma, T, NSamples, NRepl):
    dt = T / NSamples
    paths = np.zeros((NSamples + 1, NRepl))
    paths[0] = S0

    for t in range(1, NSamples + 1):
        Z = np.random.normal(0, 1, NRepl)
        paths[t] = paths[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
    
    return paths

# Corrected closed-form solution for geometric Asian call option
def geo_asian_call(S0, K, r, sigma, T, NSamples):
    dt = T / NSamples
    nu = r - 0.5 * sigma ** 2
    
    # Corrected sigma_adj and mu_adj
    sigma_adj2 = (sigma ** 2 / NSamples ** 2) * ((NSamples + 1) * (2 * NSamples + 1) / 6)
    sigma_adj = np.sqrt(sigma_adj2)
    
    mu_adj = (NSamples + 1) / (2 * NSamples) * nu + 0.5 * sigma_adj2
    
    # Compute d1 and d2
    d1 = (np.log(S0 / K) + (mu_adj + 0.5 * sigma_adj ** 2) * T) / (sigma_adj * np.sqrt(T))
    d2 = d1 - sigma_adj * np.sqrt(T)
    
    geo_price = np.exp(-r * T) * (S0 * np.exp(mu_adj * T) * st.norm.cdf(d1) - K * st.norm.cdf(d2))
    return geo_price

# Monte Carlo simulation with control variates and confidence intervals
def mc_asian_call_cv(S0, K, r, sigma, T, NSamples, NRepl):
    paths = asset_paths_gbm(S0, r, sigma, T, NSamples, NRepl)

    # Compute arithmetic and geometric averages
    arithmetic_avg = np.mean(paths[1:], axis=0)
    geometric_avg = np.exp(np.mean(np.log(paths[1:]), axis=0))

    # Compute option payoffs
    arithmetic_payoffs = np.maximum(arithmetic_avg - K, 0)
    geometric_payoffs = np.maximum(geometric_avg - K, 0)
    
    # Discounted payoffs
    X = np.exp(-r * T) * arithmetic_payoffs
    Y = np.exp(-r * T) * geometric_payoffs

    # Control variate adjustment
    E_Y = geo_asian_call(S0, K, r, sigma, T, NSamples)  # Expected value of geometric Asian option
    c_hat = np.cov(X, Y)[0, 1] / np.var(Y)  # Optimal coefficient
    X_cv = X - c_hat * (Y - E_Y)  # Adjusted estimator

    # Compute statistics
    z_score = 1.96  # 95% confidence level
    price_mc = np.mean(X)
    price_cv = np.mean(X_cv)
    
    std_error_mc = np.std(X) / np.sqrt(NRepl)
    std_error_cv = np.std(X_cv) / np.sqrt(NRepl)

    # Confidence intervals
    ci_mc = (price_mc - z_score * std_error_mc, price_mc + z_score * std_error_mc)
    ci_cv = (price_cv - z_score * std_error_cv, price_cv + z_score * std_error_cv)

    return price_mc, std_error_mc, ci_mc, price_cv, std_error_cv, ci_cv

# Set parameters
S0 = 100   # Initial stock price
K = 100    # Strike price
r = 0.05   # Risk-free rate
sigma = 0.2  # Volatility
T = 1       # Time to maturity
NSamples = 50  # Time steps
NRepl = 100000  # Number of simulations

# Run Monte Carlo with and without control variates
price_mc, std_error_mc, ci_mc, price_cv, std_error_cv, ci_cv = mc_asian_call_cv(S0, K, r, sigma, T, NSamples, NRepl)

# Print results
print(f"Monte Carlo: Price = {price_mc:.4f}, Std Error = {std_error_mc:.4f}, CI = [{ci_mc[0]:.4f}, {ci_mc[1]:.4f}]")
print(f"Control Variate: Price = {price_cv:.4f}, Std Error = {std_error_cv:.4f}, CI = [{ci_cv[0]:.4f}, {ci_cv[1]:.4f}]")
print(f"Variance Reduction Factor = {(std_error_mc / std_error_cv) ** 2:.2f}")


Monte Carlo: Price = 5.8933, Std Error = 0.0258, CI = [5.8428, 5.9437]
Control Variate: Price = 5.8571, Std Error = 0.0007, CI = [5.8557, 5.8584]
Variance Reduction Factor = 1318.12
